In [1]:
import torch
from torch.optim import Adam
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
import numpy as np
import time
from tqdm import tqdm

from ExpressionDataset import ExpressionDataset

In [2]:
device = 'cuda'
print(torch.cuda.get_device_name(0))

image_size = 96
mean = [0.5375, 0.4315, 0.3818]
std = [0.2928, 0.2656, 0.2614]

epochs = 500

NVIDIA GeForce RTX 3090 Ti


In [3]:
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

dataset = ExpressionDataset('./data/', transform=transform)
dataset = [dataset[i] for i in torch.randperm(len(dataset))]
train_len = int(len(dataset) * 0.8)
val_len = len(dataset) - train_len
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, val_len])
train_dl = DataLoader(train_dataset, batch_size=16, num_workers=8)
val_dl = DataLoader(val_dataset, batch_size=128, num_workers=8)

In [4]:
# Using a pretrained (open source) model for the sake of time
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2)
model.set_swish(memory_efficient=False)
model = model.to('cuda')

Loaded pretrained weights for efficientnet-b0


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min')

In [6]:
def train(model, loader, criterion, optimizer):
    train_loss = 0.0
    train_correct = 0

    for images, labels in tqdm(train_dl):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_correct += torch.sum(predictions == labels.data)

    return train_loss / len(train_dl)

In [7]:
from sklearn.metrics import f1_score, roc_auc_score, recall_score

def evaluate(model, loader):
    val_loss = 0.0
    val_correct = 0

    model.eval()

    label_list = []
    pred_list = []

    with torch.no_grad():
        for images, labels in tqdm(loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predictions = torch.max(outputs, 1)

            loss = criterion(outputs, labels)

            val_loss += loss.item()
            val_correct += torch.sum(predictions == labels.data)
            label_list = np.concatenate([label_list, labels.cpu()])
            pred_list = np.concatenate([pred_list, predictions.cpu()])

    val_accuracy = (100.0 * val_correct / len(loader.dataset)).cpu()
    val_f1 = f1_score(label_list, pred_list)
    val_roc_auc = roc_auc_score(label_list, pred_list)
    val_recall = recall_score(label_list, pred_list)

    return val_accuracy, val_recall, val_f1, val_roc_auc


In [9]:
train_hist = []
val_hist = []

for epoch in range(1, epochs + 1):
    s = time.time()
    loss = train(model, train_dl, criterion, optimizer)
    train_acc, train_recall, train_f1, train_roc_auc = evaluate(model, train_dl)
    val_acc, val_recall, val_f1, val_roc_auc = evaluate(model, val_dl)
    scheduler.step(loss)
    e = time.time()

    train_hist.append((train_acc, train_recall, train_f1, train_roc_auc))
    val_hist.append([val_recall])

    if sum([val_recall]) >= np.asarray(val_hist).sum(axis=1).max():
        print("Saving Best Performing Model...")
        torch.save(model.state_dict(), './models/BestModel.pt')

    print(f'Epoch: {epoch:03d}, Loss: {loss:.05f}\n'
          f'Train Acc: {train_acc}, Train Recall: {train_recall}, Train F1: {train_f1}, Train ROC-AUC: {train_roc_auc}\n'
          f'Val Acc: {val_acc}, Val Recall: {val_recall}, Val F1: {val_f1}, Val ROC-AUC: {val_roc_auc}\n'
          f'Time: {e - s:.05f} seconds')

100%|██████████| 46/46 [00:22<00:00,  2.04it/s]


Saving Best Performing Model...
Epoch: 001, Loss: 0.08416
Train Acc: 98.74317169189453, Train Recall: 0.9698914116485686, Train F1: 0.964180569185476, Train ROC-AUC: 0.9805142371834419
Val Acc: 97.21122741699219, Val Recall: 0.9112903225806451, Val F1: 0.917766497461929, Val ROC-AUC: 0.9479640319567124
Time: 150.15731 seconds


 70%|███████   | 1023/1453 [00:31<00:04, 94.11it/s]

In [ ]:
model.eval()
example_input = torch.randn(1, 3, 96, 96).cuda()

# Export the model
torch.onnx.export(model,                     # model being run
                  example_input,             # model input (or a tuple for multiple inputs)
                  "./models/model.onnx",     # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names=['input'],     # the model's input names
                  output_names=['output'],   # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})